In [2]:
import pickle
import json

def load_retrieved_data(pkl_file_path):
    with open(pkl_file_path, 'rb') as f:
        retrieved_data = pickle.load(f)
    return retrieved_data

def get_user_sequences(user_id, umap, train, val):
    if user_id not in umap:
        print(f"User ID {user_id} not found in umap.")
        return None
    user_index = umap[user_id]
    
    # Get sequences
    train_sequence = train.get(user_index, [])
    val_sequence = val.get(user_index, [])
    
    # Combine sequences
    full_sequence = train_sequence + val_sequence
    return full_sequence

def map_item_ids_to_product_info(item_ids, meta, smap):
    # Reverse smap to map item IDs to ASINs
    id_to_asin = {v: k for k, v in smap.items()}
    item_info = []
    for item_id in item_ids:
        product_name = meta.get(item_id)
        asin = id_to_asin.get(item_id)
        if product_name and asin:
            item_info.append({'item_id': item_id, 'product_name': product_name, 'asin': asin})
        else:
            print(f"Item ID {item_id} not found in meta or smap.")
    return item_info

def get_user_reviews(user_id, user_reviews_data):
    for user_data in user_reviews_data:
        if user_data['user_id'] == user_id:
            return user_data['reviews']
    print(f"User ID {user_id} not found in user reviews data.")
    return []

def match_sequences_to_reviews(item_info, user_reviews):
    # Create a set of ASINs from item_info
    sequence_asins = set([item['asin'] for item in item_info])
    
    # Find reviews where the parent_asin is in sequence_asins
    matched_reviews = []
    for review in user_reviews:
        if review['parent_asin'] in sequence_asins:
            matched_reviews.append(review)
    
    return matched_reviews

# Load data
pkl_file_path = 'data/dataset.pkl'  # Update with your actual path
retrieved_data = load_retrieved_data(pkl_file_path)

train = retrieved_data['train']
val = retrieved_data['val']
umap = retrieved_data['umap']
meta = retrieved_data['meta']
smap = retrieved_data['smap']

# User ID
user_id = 'AFSKPY37N3C43SOI5IEXEK5JSIYA'  # Replace with your user ID

# Get user sequences
user_sequence = get_user_sequences(user_id, umap, train, val)
if user_sequence is None:
    raise ValueError(f"No sequences found for user ID {user_id}")

# Map item IDs to product names and ASINs
item_info = map_item_ids_to_product_info(user_sequence, meta, smap)

# Load your user reviews JSON data
with open('input_set.json', 'r') as f:
    user_reviews_data = json.load(f)

# Get user's reviews
user_reviews = get_user_reviews(user_id, user_reviews_data)
if not user_reviews:
    raise ValueError(f"No reviews found for user ID {user_id}")

# Match sequences to reviews
matched_reviews = match_sequences_to_reviews(item_info, user_reviews)

# Display matched reviews
print(f"Matched reviews for user {user_id}:")
for review in matched_reviews:
    print(f"- {review['product_name']} (ASIN: {review['parent_asin']}) (review: {review['text']})")


Matched reviews for user AFSKPY37N3C43SOI5IEXEK5JSIYA:
- Manicure and Pedicure Nail Clipper from POWERGROOMING - Powerful Trimmer for Thick and Thin Finger Nails and Toe Nails - Included Nail File and"Catcher" for Easy Cleanup (1 Pack) (ASIN: B07J3GH1W1) (review: This a really cute kit which would make for a great gift for someone. It is in a little leather like pouch and has everything you need to give yourself a quality manicure. The nail clipper is a perfect size and works just as well on a women or man's nails. The file is nice as well (although I still prefer to use emory boards on mine). I actually bought another one of these to give to my son as a stocking stuffer this last Christmas for him to use at college. Just a nice, quality made kit at a reasonable price.)
- Iryasa Night Indulge Cream - Natural Face Cream for Dry Skin - Vegan Anti Aging Night Cream for Women - Firming Cream for Face and Neck - Organic Vitamin C Moisturizer for Face - 1.7oz (ASIN: B07W397QG4) (review: To b